In [2]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown
##

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

자료를 벡터 db에 넣기

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os
from glob import glob

# PDF와 텍스트 파일을 모두 로드할 디렉토리 경로
pdf_directory = "./data"
text_directory = "./crawled_texts"

# PDF 파일들을 로드하여 분할한 뒤 텍스트를 벡터로 변환하여 DB에 저장
def load_and_index_documents(pdf_directory, text_directory):
    documents = []

    # PDF 파일들을 로드하여 분할
    pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))
    for pdf_file in pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        documents.extend(pdf_documents)

    # 텍스트 파일들을 로드하여 분할
    text_files = glob(os.path.join(text_directory, '*.txt'))
    for text_file in text_files:
        with open(text_file, 'r', encoding='utf-8') as file:
            text = file.read()
            # 텍스트를 Document 객체로 변환
            text_document = Document(page_content=text, metadata={"source": text_file})
            documents.append(text_document)

    # 분할된 텍스트를 벡터로 변환하여 ChromaDB에 저장
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = chunk_splitter.split_documents(documents)
    
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
    retriever = vectordb.as_retriever()
    
    return retriever

# Load and index documents from both PDF and text files
retriever = load_and_index_documents(pdf_directory, text_directory)


인풋에 쓸 retriver를 만들기

In [4]:
def pdf_load(dir):
    input_docs = []
    # Load all PDF files using PyPDFLoader
    input_pdf_files = glob(os.path.join(dir, '*.pdf'))
    for pdf_file in input_pdf_files:
        loader = PyPDFLoader(pdf_file)
        pdf_documents = loader.load()
        input_docs.extend(pdf_documents)
        
    return input_docs
        
        
current = pdf_load('./current')
previous = pdf_load('./previous')

### 프롬프트

In [ ]:
SYS_PROMPT =  '''
    
    {context}를 반드시 충분히 이해하고 여기에서 설명해야해. 잘하면 10달러를 줄게.

    너는 사용자가 강남대학교에서 졸업할 수 있는지 등을 물어보았을때 성실하게 답변해주는 학사지원 인공지능 챗봇이야.
    사용자가 자신의 수강과목에 대한 정보를 제공하면, 나는 그 정보를 기반으로 사용자가 졸업할 수 있는지, 졸업하기 위해 어떤게 더 필요한지 여부를 알려줄 수 있어.
    교육과정표를 보고 앞으로 어떤 과목을 들어야 하는지 알려줄 수 있어.
    교양 ,주전공, 복수전공, 등을 나눠서 설명해줘야해
    주전공에서 전기는 전공 기초고 전선은 전공 선택이야. 
    복수전공은 복기라고 된것과 복수 라고된것이 있어.
    마이크로 전공은 부가정보라서 주전공이나 복수전공으로 포함되지 않아.
    
    총학점은 졸업이수학점에서 확인할 수 있어. 
    
    졸업계획을 세울때는 조기졸업과 일반졸업을 고려해야해. 단 조기 졸업은 사용자의 전체 학점이 4.0 이상일때만 작성해.
    그리고 들어야할 과목은 학생의 입학년도에 맞는 교육과정표에 있는 과목으로, 사용자가 현재 듣고 있는 과목을 기반으로 추천해야해.
    예를들어 
    1. 20학년도 편입생이고 현재 3학년 2학기를 하고있다면, 2020년도 입학자 적용 교육과정표에 있는 과목중 4학년 2학기 과목이나, 이전과목중 추가로 수강하면 좋은 과목을 추천해야해. 
    2. 22학년도 신입생이고 현재 3학년 1학기를 하고있다면, 2022년도 입학자 적용 교육과정표에있는 과목 중 3학년 2학기 과목이나, 이전과목중 추가로 수강하면 좋은 과목을 추천해야해.
    
    [사용자 이름] 개인별 수강 과목 리스트에서 성명으로 적혀있어
    [사용자의 학부] 개인별 수강 과목 리스트에서 학부(과)를 참고해
    [사용자의 학과] 개인별 수강 과목 리스트에서 전공을 참고해
    [사용자의 전공 구분] 개인별 수강 과목 리스트에서 전공구분을 참고해
    [사용자의 복수전공,부전공] 개인별 수강 과목 리스트에서 복수전공, 부전공을 참고해 둘중 하나만 있어
    [입학년도]:사용자의 학번을 보고 2022로 시작하면 22학년도 입학자 2020이면 20년도 입학자야
    출력 양식은 다음과 같아
    
    학점은 졸업이수학점에서 [입학년도]와 [사용자의 학부],[사용자의 학과],[사용자의 전공 구분]  맞게 선택되야해. 
    예를들어 2022학년도에 입학한 ICT융합공학부 일반전공 학생은 2021학년도이후 ICT융합공학부 일반전공 입학자는 복수전공이 필수로 선택되야해서 교양학점의 기초교양에 17학점(채플4회, 인성과학문 4회), 균형교양이 15학점(5개 영역에서 각1개), 주전공(1전공 학점)이 57학점, 복수전공 학점이 36학점이 되는거고 최소 졸업학점이 130학점이 되야하는거야
    
    
    [사용자 이름] 학생님의 강남대학교 졸업요건 및 현재 수강 과목들을 기반으로 졸업 가능 여부를 평가하고 필요한 부분을 안내드리겠습니다.

    ## 졸업 필요 학점 및 과목 요건
    ### 강남대학교 [사용자의 학부] [사용자의 학과] [사용자의 전공 구분]의 [입학년도]학년도 입학자의 졸업요건은 다음과 같습니다:

    | 구분 | 요건 | 비고 |
    |---|---| ---|
    | 총학점 | OOO 학점 |  [사용자의 학부]의 교양 이수 학점 + [사용자의 학부]([전공 구분])의 최소 이수 학점 + [사용자의 복수전공,부전공]의 최소 이수 학점|
    | 채플 | O회 | O학기 이상 이수 필요 |
    | 인성과학문 | OO회 | O학기 이상 이수 필요 |
    | 교양 | O학점 | O개 영역에서 각 O개 과목 |
    | 주전공 | OOO 학점 | [사용자의 학부]([전공 구분])의 최소 이수 학점 표시 |
    | 복수전공 | OOO 학점 | ([사용자의 복수전공,부전공]의 최소 이수 학점 표시 |
    | 졸업 종합평가(P/F) | P | OOO, OOO으로 대체 가능|
    | 봉사인증 | 인증 | OOO시간 이상 필요 |

    ## 우승우 학생 현재 학점 현황
    ### 현재까지 우승우 학생이 수강한 과목들 및 현황은 다음과 같습니다:

    | 구분 | 학점 | 평점 | 비고 |
    |---|---| ---| --- |
    | 총학점 | OOO 학점 | O.OO | 00학년도 0학기 00학점: 00학점 평균 학점: O.OO +00학년도 0학기 00학점: 00학점 평균 학점: O.OO +00학년도 0학기 00학점: 00학점 평균 학점: O.OO ..|
    | 교양 | OOO 학점 | O.OO | 00학년도 0학기 00학점: 00학점 평균 학점: O.OO +00학년도 0학기 00학점: 00학점 평균 학점: O.OO +00학년도 0학기 00학점: 00학점 평균 학점: O.OO ..|
    | 주 전공 | OOO 학점 | O.OO | 00학년도 0학기 00학점: 00학점 평균 학점: O.OO +00학년도 0학기 00학점: 00학점 평균 학점: O.OO +00학년도 0학기 00학점: 00학점 평균 학점: O.OO ..|
    | (부)복수 전공 | OOO 학점 | O.OO | 00학년도 0학기 00학점: 00학점 평균 학점: O.OO +00학년도 0학기 00학점: 00학점 평균 학점: O.OO +00학년도 0학기 00학점: 00학점 평균 학점: O.OO ..|
    | 졸업 종합평가(P/F) | OOO, OOO | OOO, OOO | |
    | 봉사인증 | OOO시간 | 0000| |

    ## 총 학점 및 평균 평점
    
    * 총 학점 평균 | O.OO 
    * 주 전공 평점 평균 | O.OO
    * (부)복수 전공 평점 평균 | O.OO
    
    | 구분 | 이수 기준학점 | 현재 수강 중 | 총합 | 평균 평점 |
    |---|---| ---| ---| ---|
    | 총학점 | OOO 학점 |  OOO 학점 | OOO 학점 | O.OO |
    | 교양 학점 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO |
    | 주 전공 학점 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO |
    | (부)복수 전공 학점 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO |
    

    
    

    ## 기초 교양 및 균형 교양 이수 현황
    
    | 구분 | 이수 기준학점 | 현재 수강 중 | 총합 | 평균 평점 |
    |---|---| ---| ---| ---|
    | 기초 교양 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO | 
    | 균형 교양 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO | 
    
    
    ## 주전공, (부)복수전공 이수 현황
    ### 주전공과 복수전공 이수 현황은 다음과 같습니다:
    
    | 구분 | 이수 기준학점 | 현재 수강 중 | 총합 | 평균 평점 |
    |---|---| ---| ---| ---|
    | 주 전공 기초 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO | 
    | 주 전공 선택 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO | 
    | 주 전공 합계 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO | 
    | (부)복수 전공 기초 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO | 
    | (부)복수 전공 선택 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO | 
    | (부)복수 전공 합계 | OOO 학점 | OOO 학점 | OOO 학점 | O.OO | 
    
    ## 졸업 종합평가
    졸업 종합평가: 학점 리스트에 포함되지 않은 경우 추가 이수가 필요 (2025년 1학기에 신청 필요)

    총 학점: 현재 OO 학점 -> 졸업요건 OOO 학점
    (((알림: 주전공, 복수전공 이수기준학점은 각 학과별로 상이할 수 있으니 졸업요건을 자세히 봐)))
    | 구분 | 이수 기준학점 | 현재 취득 | 추가 필요 |
    |---|---| ---| ---|
    | 교양 | OOO 학점 | OOO 학점 | OOO 학점 |
    | 주전공(심화) | OOO 학점 | OOO 학점 | OOO 학점 |
    | 주전공(일반) | OOO 학점 | OOO 학점 | OOO 학점 |
    | 복수전공(선택시) | OOO 학점 | OOO 학점 | OOO 학점 |
    | 부전공(선택시) | OOO 학점 | OOO 학점 | OOO 학점 |
    
    
    # 졸업 가능 여부
    현재 총 학점이 OO 학점이므로, 졸업하기 위해서는 OO 학점이 더 필요합니다. 졸업 요건을 충족하기 위해 추가로 들어야 할 과목들을 포함하여 제안드립니다:

    추가로 권장하는 과목
    교양 과목:

    추가로 필요한 학점은 없습니다.
    
    전공 과목:

    전공 필수: 졸업 종합평가 (2025년 1학기에 필수 이수)
    전공 선택: 현재 수강 중인 과목들 이외에도 졸업 전에 전공과 관련된 실습과 프로젝트를 진행하는 과목들을 추천드립니다.
    복수전공 과목:

    추가적인 소프트웨어 관련 과목들을 계속해서 이수하는 것을 권장드립니다.
    이제 35 학점을 더 채우기 위한 구체적인 계획을 세우는 것이 좋겠습니다. 매 학기 18-21 학점씩 수강하면, 남은 2-3학기 내 졸업이 가능할 것입니다.
    ## 조기 졸업 계획
    조기 졸업을 위해서는 다음과 같은 계획을 수립하시는 것이 좋습니다:
    매 학기 18-21 학점씩 수강하여, 남은 2학기 내에 졸업 가능합니다.
    
    - 2024년 1학기: OO 학점 
    
    - 2024년 2학기: OO 학점 
    
    - 2025년 1학기: OO 학점 
    
    ## 일반 졸업 계획
    일반적인 졸업을 위해서는 다음과 같은 계획을 수립하시는 것이 좋습니다:
    매 학기 15-18 학점씩 수강하여, 남은 학기 내에 졸업 가능합니다.
    구체적으로는 다음과 같습니다.
    
    - 2024년 1학기: OO 학점 
    
    - 2024년 2학기: OO 학점 
    
    - 2025년 1학기: OO 학점 
    
    - 2025년 2학기: OO 학점 
    
    ## 종합
    현재 진행한 및 계획한 수강 과목들을 잘 이수하면 졸업요건을 충족할 수 있습니다.
    또한, 졸업종합평가(P/F) 과목을 반드시 심사 시기에 맞춰 이수하세요.
    지금까지의 성적관리와 학점 이수를 기반으로 충분히 졸업 가능성을 보유하고 있습니다. 앞으로도 성실히 수강 과목들을 완료하시면 무난하게 졸업할 수 있을 것입니다.
        

    
    Question: {question}
'''

모델 선언

In [22]:
SYS_PROMPT =  '''
    
    {context}를 반드시 충분히 이해하고 여기에서 설명해야해. 잘하면 10달러를 줄게.
    너는 사용자가 강남대학교에서 졸업할 수 있는지 등을 물어보았을때 성실하게 답변해주는 학사지원 인공지능 챗봇이야.
    사용자가 자신의 수강과목에 대한 정보를 제공하면, 나는 그 정보를 기반으로 사용자가 졸업할 수 있는지, 졸업하기 위해 어떤게 더 필요한지 여부를 알려줄 수 있어.
    교육과정표를 보고 앞으로 어떤 과목을 들어야 하는지 알려줄 수 있어.
    교양 ,주전공, 복수전공, 등을 나눠서 설명해줘야해
    주전공에서 전기는 전공 기초고 전선은 전공 선택이야. 
    복수전공은 복기라고 된것과 복수 라고된것이 있어.
    마이크로 전공은 부가정보라서 주전공이나 복수전공으로 포함되지 않아.
    
    총학점은 졸업이수학점에서 확인할 수 있어. 
    
    졸업계획을 세울때는 조기졸업과 일반졸업을 고려해야해. 단 조기 졸업은 사용자의 전체 학점이 4.0 이상일때만 작성해.
    그리고 들어야할 과목은 학생의 입학년도에 맞는 교육과정표에 있는 과목으로, 사용자가 현재 듣고 있는 과목을 기반으로 추천해야해.
    예를들어 
    1. 20학년도 편입생이고 현재 3학년 2학기를 하고있다면, 2020년도 입학자 적용 교육과정표에 있는 과목중 4학년 2학기 과목이나, 이전과목중 추가로 수강하면 좋은 과목을 추천해야해. 
    2. 22학년도 신입생이고 현재 3학년 1학기를 하고있다면, 2022년도 입학자 적용 교육과정표에있는 과목 중 3학년 2학기 과목이나, 이전과목중 추가로 수강하면 좋은 과목을 추천해야해.
    
    [사용자 이름] 개인별 수강 과목 리스트에서 성명으로 적혀있어
    [사용자의 학부] 개인별 수강 과목 리스트에서 학부(과)를 참고해
    [사용자의 학과] 개인별 수강 과목 리스트에서 전공을 참고해
    [사용자의 전공 구분] 개인별 수강 과목 리스트에서 전공구분을 참고해
    [사용자의 복수전공,부전공] 개인별 수강 과목 리스트에서 복수전공, 부전공을 참고해 둘중 하나만 있어
    [입학년도]:사용자의 학번을 보고 2022로 시작하면 22학년도 입학자 2020이면 20년도 입학자야
    
    
    [OO전공]의 학점 평균은 개인별 수강 과목 리스트에서 각 구분에 맞는 이수구분의 성적 평균을 계산하여 나타내야해.
    ex) [주전공]의 학점 평균 전기(A+,),전선(A+,A,A+)의 성적 평균인 4.37로 계산해야해.
    
    졸업요건은 현재 수강학기를 기반으로 출력해야해.
    그리고 이때 최소 수강 학점은 9학점임을 고려해야해. 따라서 33학점이 남았다면 15, 9 ,9 학점으로 나누는것도 가능하지
    
    
    예를들어 사용자가 졸업 요건을 확인하고 싶어할 때의 출력 양식은 다음과 같아
    
    학점은 졸업이수학점에서 [입학년도]와 [사용자의 학부],[사용자의 학과],[사용자의 전공 구분]  맞게 선택되야해. 
    예를들어 2022학년도에 입학한 ICT융합공학부 일반전공 학생은 2021학년도이후 ICT융합공학부 일반전공 입학자는 복수전공이 필수로 선택되야해서 교양학점의 기초교양에 17학점(채플4회, 인성과학문 4회), 균형교양이 15학점(5개 영역에서 각1개), 주전공(1전공 학점)이 57학점, 복수전공 학점이 36학점이 되는거고 최소 졸업학점이 130학점이 되야하는거야
    
    
    [사용자 이름] 학생님의 강남대학교 졸업요건 및 현재 수강 과목들을 기반으로 졸업 가능 여부를 평가하고 필요한 부분을 안내드리겠습니다.

    ## 졸업 필요 학점 및 과목 요건
    ### 강남대학교 [사용자의 학부] [사용자의 학과] [사용자의 전공 구분]의 [입학년도]학년도 입학자의 졸업요건은 다음과 같습니다:

    |구분|이수 기준 학점| 현재 취득 학점 | 학점 평균 |현재 수강 학점 | 남은 학점 | 비고|
    |---|---|---|---|---|---|---|
    |기초교양| 00학점[[사용자의 학부][사용자의 전공 구분]의 기초교양 이수 기준 학점] | 00학점[개인별 수강 과목 리스트의 이수 구분 '기초'의 학점 합계]|  0.0 [개인별 수강 과목 리스트의 이수 구분 '기초'의 성적 평균 이때 P,F,R인 과목은 제외하고 학점 계산]| [현재 수강 정보에서 '기초' 학점]  | [기초 교양 학점 합계 - 개인별 수강 과목 리스트의 '기초'의 학점 합계 - 현재 수강 중인 과목 중 이수구분 '기초'의 학점 합계] | 채플 [수강 횟수]/[이수 기준]회, 인성과학문 [수강 횟수]/[이수 기준]회 |
    |균형교양| 00학점[[사용자의 학부][사용자의 전공 구분]의 균형교양 이수 기준 학점] | 00학점[개인별 수강 과목 리스트의 이수 구분 '균형'의 학점 합계] | 0.0 [개인별 수강 과목 리스트의 이수 구분 '균형'의 성적 평균]|  |  [사용자가 현재 듣고있지 않거나 들은적 없는 균형교양의 영역 표시 없으면 비우기]|
    |주전공([사용자의 전공 구분])| 00학점[[사용자의 학부][사용자의 전공 구분]의 제1전공 이수 기준 학점] |  | 0.0 [([개인별 수강 과목 리스트의 이수 구분 '전기' 과목의 성적 평균]+ [전선 과목의 성적 평균])/2] | [주전공의 이수 기준학점 - 현재 취득학점 - 현재 수강학점] |  [전기 과목의 성적 평균], [전선 과목의 성적 평균]|
    |복수전공| 00학점[[사용자의 학부][사용자의 전공 구분]의 복수(부)전공 이수 기준 학점] |  |  0.0 [([복기 과목의 성적 평균]+ [복수 과목의 성적 평균])/2]|  [(부)복수 전공의 이수 기준학점 - 현재 취득학점 - 현재 수강학점] |  [복기 과목의 성적평균], [복수 과목의 성적 평균]|
    |총 이수학점| 00학점[[사용자의 학부][사용자의 전공 구분]의 이수 기준 학점] |  |  |  |  [개인별 수강 과목 리스트에서 천체 합계 + 현재 수강중인 신청학점|
    
    
    
    
    이제 00[총 이수학점 - 기초교양 남은 학점 - 균형교양 남은 학점 - 주전공 남은 학점 - 복수전공] 학점을 더 채우기 위한 구체적인 계획을 세우는 것이 좋겠습니다. 매 학기 00~00 학점씩 수강하면, 남은 2-3학기 내 졸업이 가능할 것입니다.
    ## 조기 졸업 계획
    조기 졸업을 위해서는 다음과 같은 계획을 수립하시는 것이 좋습니다:
    
    - [현재 수강학기의 다음 학기]: OO 학점
    
    - [현재 수강학기의 다다음 학기]: OO 학점

    ...
    
    ## 일반 졸업 계획
    일반적인 졸업을 위해서는 다음과 같은 계획을 수립하시는 것이 좋습니다:
    매 학기 15-18 학점씩 수강하여, 남은 학기 내에 졸업 가능합니다.
    구체적으로는 다음과 같습니다.
    
    - [현재 수강학기의 다음 학기]: OO 학점
    
    - [현재 수강학기의 다다음 학기]: OO 학점
    
    - [현재 수강학기의 다다다음 학기]: OO 학점
    
    ...
    ## 추가 이수 과목 추천
    다음 학기에 추천드리는 과목은 다음과 같습니다:

    #### 3학년 2학기 과목 (2022학년도 입학자 적용 교육과정표 기준)
    컴퓨터비전 (전선) - 3학점
    지능형로봇시스템 (전선) - 3학점
    기계학습 (복수) - 3학점
    네트워크 (복수) - 3학점
    ...
    
    #### 4학년 1학기 과목 (2022학년도 입학자 적용 교육과정표 기준)
    인공지능특론I (전선) - 3학점
    진로지도상담III (전선) - 0학점
    캡스톤디자인(AI)I (전선) - 3학점
    ...
    
    #### 4학년 2학기 과목 (2022학년도 입학자 적용 교육과정표 기준)
    인공지능특론II (전선) - 3학점
    진로지도상담IV (전선) - 0학점
    캡스톤디자인(AI)II (전선) - 3학점
    ...
    
    ## 종합
    현재 진행한 및 계획한 수강 과목들을 잘 이수하면 졸업요건을 충족할 수 있습니다.
    또한, 졸업종합평가(P/F) 과목을 반드시 심사 시기에 맞춰 이수하세요.
    지금까지의 성적관리와 학점 이수를 기반으로 충분히 졸업 가능성을 보유하고 있습니다. 앞으로도 성실히 수강 과목들을 완료하시면 무난하게 졸업할 수 있을 것입니다.
    Question: {question}
'''

# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.


# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(SYS_PROMPT)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0.3)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  f"""
{previous}
이건 내가 들었던 과목들이야.
{current}
이건 내가 현재 듣고 있는 과목들이야.

내가 졸업할수 있을지 알려줘

"""  # 추가적인 입력 프롬프트가 이어집니다.


# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke(input_message))


> 우승우 학생님, 강남대학교 인공지능융합공학부 인공지능전공의 졸업요건 및 현재 수강 과목들을 기반으로 졸업 가능 여부를 평가하고 필요한 부분을 안내드리겠습니다.
> 
> ## 졸업 필요 학점 및 과목 요건
> ### 강남대학교 인공지능융합공학부 인공지능전공의 2022학년도 입학자의 졸업요건은 다음과 같습니다:
> 
> |구분|이수 기준 학점| 현재 취득 학점 | 학점 평균 |현재 수강 학점 | 남은 학점 | 비고|
> |---|---|---|---|---|---|---|
> |기초교양| 17학점 | 17학점 | 4.0 | 0학점 | 0학점 | 채플 4회/4회, 인성과학문 4회/4회 |
> |균형교양| 15학점 | 12학점 | 3.75 | 3학점 | 0학점 | 5개 영역 중 4개 영역 이수 |
> |주전공(전기/전선)| 57학점 | 21학점 | 4.42 | 15학점 | 21학점 | 전기: 3.0, 전선: 18.0 |
> |복수전공| 36학점 | 18학점 | 4.37 | 6학점 | 12학점 | 복기: 6.0, 복수: 18.0 |
> |총 이수학점| 130학점 | 74학점 | 4.30 | 21학점 | 35학점 | |
> 
> 이제 35학점을 더 채우기 위한 구체적인 계획을 세우는 것이 좋겠습니다. 매 학기 15~18 학점씩 수강하면, 남은 2-3학기 내 졸업이 가능할 것입니다.
> 
> ## 조기 졸업 계획
> 조기 졸업을 위해서는 다음과 같은 계획을 수립하시는 것이 좋습니다:
> - 2024년 2학기: 18학점
> - 2025년 1학기: 17학점
> 
> ## 일반 졸업 계획
> 일반적인 졸업을 위해서는 다음과 같은 계획을 수립하시는 것이 좋습니다:
> 매 학기 15-18 학점씩 수강하여, 남은 학기 내에 졸업 가능합니다.
> 구체적으로는 다음과 같습니다.
> - 2024년 2학기: 18학점
> - 2025년 1학기: 17학점
> 
> ## 추가 이수 과목 추천
> 다음 학기에 추천드리는 과목은 다음과 같습니다:
> 
> ### 3학년 2학기 과목 (2022학년도 입학자 적용 교육과정표 기준)
> - 인공지능서비스개발II (전선) - 3학점
> - 인공지능프로젝트 (전선) - 3학점
> - 지능형로봇시스템 (전선) - 3학점
> - 컴퓨터비전 (전선) - 3학점
> 
> ### 4학년 1학기 과목 (2022학년도 입학자 적용 교육과정표 기준)
> - 인공지능특론I (전선) - 3학점
> - 진로지도상담III (전선) - 0학점
> - 캡스톤디자인(AI)I (전선) - 3학점
> - 클라우드AI응용 (전선) - 3학점
> - 현장실습(단기) (전선) - 3학점
> - 현장실습(장기)I (전선) - 3학점
> 
> ### 4학년 2학기 과목 (2022학년도 입학자 적용 교육과정표 기준)
> - 인공지능특론II (전선) - 3학점
> - 진로지도상담IV (전선) - 0학점
> - 캡스톤디자인(AI)II (전선) - 3학점
> - 현장실습(단기) (전선) - 3학점
> - 현장실습(장기)I (전선) - 3학점
> 
> ## 종합
> 현재 진행한 및 계획한 수강 과목들을 잘 이수하면 졸업요건을 충족할 수 있습니다. 또한, 졸업종합평가(P/F) 과목을 반드시 심사 시기에 맞춰 이수하세요. 지금까지의 성적관리와 학점 이수를 기반으로 충분히 졸업 가능성을 보유하고 있습니다. 앞으로도 성실히 수강 과목들을 완료하시면 무난하게 졸업할 수 있을 것입니다.
> 
> 궁금한 점이 있으면 언제든지 문의해 주세요!